In [1]:
# Scinet modules
import scinet.ed_copernicus_elliptic as edc
from scinet import *

# My custom modules
import scinet.ed_stokes as sto

#Other modules
import datetime
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import numpy as np

%load_ext tensorboard
import tensorflow as tf
import datetime, os

import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard

### Plotting

In [2]:
def fix_jumps(theta_M):
    """
    Fixes jumps that arise because theta_M is always between -pi and pi
    """
    while True:
        diff = np.abs(theta_M[:, 1:] - theta_M[:, :-1])
        jumps = np.array(np.where(diff > 1.)).T
        if len(jumps) == 0:
            break
        fixed_lines = []
        for x, y in jumps:
            if x in fixed_lines:
                continue
            else:
                fixed_lines.append(x)
            theta_M[x, y + 1:] = theta_M[x, y + 1:] - np.sign(theta_M[x, y + 1] - theta_M[x, y]) * 2 * np.pi
    return theta_M

In [3]:
def copernicus_latent_neurons(net, series_length=50, delta_t=25, steps=50):
    layer = net.state_means
    T_earth = 365.26   
    T_mars = 686.97959
    # the variable eps removes a small window from the plot, such that the discontinuity is not shown
    eps = 0.1
    ss = np.linspace(- T_earth/2 + eps, T_earth/2 - eps, num=steps)
    mm = np.linspace(- T_mars/2 + eps, T_mars/2 - eps, num=steps)
    S, M = np.meshgrid(ss, mm)
    data = edc.copernicus_data(series_length, delta_t=delta_t, t_earth_target=np.ravel(S), t_mars_target=np.ravel(M))[0]
    fig = plt.figure(figsize=(6.0, 2.8))
    fig.tight_layout()
    out = np.array(net.run(data, layer))
    for i in range(len(out[0])):
        zs = out[:, i]
        ax = fig.add_subplot('12{}'.format(i+1), projection='3d')
        ax.view_init(20, 60)
        Z = np.reshape(zs, S.shape)
        surf = ax.plot_surface(S, M, Z, rstride=1, cstride=1, cmap=cm.inferno, linewidth=0)
        ax.set_xlabel(r'$M_E$')
        ax.set_ylabel(r'$M_M$')
        ax.set_zlabel('Latent activation {}'.format(i + 1))
        ax.set_xticks([- T_earth/2.+ eps, 0, T_earth/2.- eps])
        ax.set_yticks([- T_mars/2.+ eps,0, T_mars/2- eps])
        ax.set_xticklabels(['$-\pi$', r'0', r'$\pi$'])
        ax.set_yticklabels(['$-\pi$', r'0', r'$\pi$'])
    return fig

In [4]:
def get_coordinates(phi, a, ecc):    
    r = edc.get_radius(phi,a,ecc)
    return r * np.cos(phi), r * np.sin(phi)

def plot_orbits(series_length=100, delta_t = 7): 
    AU = 149597870700       
    a_mars = 1.52366231 * AU   
    a_earth = 1.00000011 * AU  
    ecc_earth = 0.01671022 
    ecc_mars = 0.09341233  
    phi_series= edc.copernicus_data(series_length, 1, delta_t = delta_t)[1][0]
    x_earth = [get_coordinates(phi_series[i][0], a_earth, ecc_earth)[0] for i in range(len(phi_series))]
    y_earth = [get_coordinates(phi_series[i][0], a_earth, ecc_earth)[1] for i in range(len(phi_series))]
    x_mars = [get_coordinates(phi_series[i][1], a_mars, ecc_mars)[0] for i in range(len(phi_series))]
    y_mars = [get_coordinates(phi_series[i][1], a_mars, ecc_mars)[1] for i in range(len(phi_series))]
    f, ax = plt.subplots(figsize=(6.0, 6.0))
    ax.plot(x_earth,y_earth)
    ax.plot(x_mars,y_mars)
    ax.set_xlabel(r'$x$ [m]')
    ax.set_ylabel(r'$y$ [m]')
    
def plot_velocity_mars(series_length=100, delta_t=7):
    AU = 149597870700       
    a_mars = 1.52366231 * AU     
    ecc_mars = 0.09341233   
    phi_series= edc.copernicus_data(series_length, 1, delta_t = delta_t)[1][0]
    phi_mars = [phi_series[i][1] for i in range(len(phi_series)-1)]
    dx_mars = [get_coordinates(phi_series[i+1][1], a_mars, ecc_mars)[0] - get_coordinates(phi_series[i][1], a_mars, ecc_mars)[0] for i in range(len(phi_series)-1)]
    dy_mars = [get_coordinates(phi_series[i+1][1], a_mars, ecc_mars)[1] - get_coordinates(phi_series[i][1], a_mars, ecc_mars)[1] for i in range(len(phi_series)-1)]
    v_mars = [(dx_mars[i] ** 2 + dy_mars[i] ** 2)**(1./2)/(delta_t*24.*3600.) for i in range(len(phi_series)-1)]
    f, ax = plt.subplots(figsize=(6.0, 6.0))
    ax.plot(phi_mars,v_mars)
    ax.set_xlabel(r'true anomaly [rad]')
    ax.set_ylabel(r'velocity [m/s]')

## Load pre-trained model

### Parameters
- `latent_size: 2`
- `input_size: 1` 
- `output_size: 2`
- other parameters: default values
### Data
- Training data: 95000 samples with `delta_t: 25`
- Validation data: 5000 samples with `delta_t: 25`

### Training
1. 2000 epochs with `batch_size: 256`, `learning_rate: 1e-3`, `beta: 0.005`, `euler_l2_coeff: 1`, `time_series_length: 20`
2. 1000 epochs with `batch_size: 1024`, `learning_rate: 1e-3`, `beta: 0.005`, `euler_l2_coeff: 1`, `time_series_length: 50`
3. 250 epochs with `batch_size: 1024`, `learning_rate: 3e-3`, `beta: 0.01`, `euler_l2_coeff: 1`, `time_series_length: 50`
4. 750 epochs with `batch_size: 250`, `learning_rate: 3e-3`, `beta: 0.01`, `euler_l2_coeff: 1`, `time_series_length: 50`
5. 1000 epochs with `batch_size: 1024`, `learning_rate: .5e-3`, `beta: 0.005`, `euler_l2_coeff: 1`, `time_series_length: 50`

In [5]:
net_pretrained = nn.Network.from_saved('copernicus_elliptic')

{'decoder_num_units': [100, 100], 'tot_epochs': 5001, 'latent_size': 2, 'output_size': 2, 'time_series_length': 50, 'encoder_num_units': [100, 100], 'euler_num_units': [], 'input_size': 1, 'load_file': 'copernicus_elliptic', 'name': 'net_copernicus_elliptic_20'}







Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where








INFO:tensorflow:Restoring parameters from /home/jrwest/Courses/Winter_2021/CS230/project/nn_physical_concepts_cs230/tf_save/copernicus_elliptic.ckpt
Loaded network from file copernicus_elliptic


In [9]:
print(net_pretrained.euler_num_units)

[]


## New Network

In [53]:
netName = 'CoperNetcus'
observation_size = 2
latent_size = 2
question_size = 0
answer_size = 2
dev_percent = 5
num_examples = 100000
time_series_length = 20

encoder_layout = [100,100]
decoder_layout = [100,100]
timestep_layout = []
myBeta = 5e-3
myL2_coeff = 1.0

### Data creation and loading

In [45]:
edc.copernicus_data(time_series_length, num_examples, fileName='Copernicus_example');

In [46]:
td, vd, ts, vs = dl.load(dev_percent, 'Copernicus_example')

In [71]:
print(np.shape(vd))
print(np.shape(td))

(5000, 40)
(95000, 40)


### Create and train neural network

In [51]:
# Create network object
net = nn.Network(latent_size, observation_size, time_series_length, answer_size,
                 encoder_num_units=encoder_layout, decoder_num_units=decoder_layout, euler_num_units=timestep_layout,
                 name=netName) 

In [73]:
# Print initial reconstruction loss (depends on initialization)
print(net.run(vd, net.recon_loss)) #default
print(net.run(vd, net.kl_loss))
print(net.rnn_depth)

#print(net_pretrained.run(vd, net_pretrained.recon_loss)) #default
#print(net_pretrained.run(vd, net_pretrained.kl_loss))
#print(net_pretrained.rnn_depth)

0.0031878161
6.6530795
18


In [30]:
log_path = os.path.join(io.tf_log_path, netName)
%load_ext tensorboard
%tensorboard --logdir /home/jrwest/Courses/Winter_2021/CS230/project/nn_physical_concepts_cs230/tf_log/StokesNet

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [56]:
# Train
num_epochs = 400
print_frequency = 0.2

check_epochs = int(print_frequency * num_epochs)
num_loops    = int(num_epochs/check_epochs)

for i in range(num_loops):
    net.train(check_epochs, 256, 0.001, td, vd, beta_fun=(lambda x: myBeta), euler_l2_coeff=myL2_coeff, test_step=10 )
    # Check progress. It is recommended to use Tensorboard instead for this.
    print(net.run(vd, net.recon_loss)) #default
    print(net.run(vd, net.kl_loss))

  0%|          | 0/1 [00:00<?, ?it/s]

0.000975271
6.709429


  0%|          | 0/1 [00:00<?, ?it/s]

0.0011996512
6.6395516


  0%|          | 0/1 [00:00<?, ?it/s]

0.0041250545
6.5788608


  0%|          | 0/1 [00:00<?, ?it/s]

0.0047628605
6.7896576


  0%|          | 0/1 [00:00<?, ?it/s]

0.0031878161
6.6530795


## Plot elliptical orbits and velocity of Mars

In [17]:
%matplotlib tk
plot_velocity_mars(series_length=1000, delta_t=.7)

## Plot of latent activations

In [58]:
# The mean anomalies M_E and M_M of Earth and Mars, respectively.
%matplotlib tk
fig = copernicus_latent_neurons(net,series_length = 20)

### Calculate L2-norm of error

In [64]:
data, states = edc.copernicus_data(20, 5000)

In [65]:
np.sqrt(net.run(data, net.recon_loss))

0.05677938

In [29]:
example, example_states = edc.copernicus_data(50, 1)
out = net.run(example, net.decoded_list)
out = np.reshape(out, (len(out),2))
plot_orbits(series_length=100, delta_t = 7)

In [66]:
#Choose Network Name for saving (if desired)
date_str  = str(datetime.datetime.now().date())
name_str  = 'CoperNetcus_'
filename  = name_str + date_str
full_path = io.tf_save_path + filename

In [67]:
#Save Network
if os.path.isfile(full_path+'.pkl'):
    print("Filename already exists. Please choose another name.")
else:    
    net.save(filename)

Saved network to file CoperNetcus_2021-03-10
